In [1]:
%run ../Python_files/util.py

No dicts found; please check load_dicts...


In [11]:
with open('./benchmark_data/Anaheim_flow.txt') as Anaheim_flow:
    Anaheim_flow_lines = Anaheim_flow.readlines()
Anaheim_links = []
i = -6
for line in Anaheim_flow_lines:
    i += 1
    if i > 0:
        Anaheim_links.append(line.split('\t')[1:3])
numLinks = i

In [20]:
numLinks

914

In [21]:
numNodes = max([int(Anaheim_links[i][1]) for i in range(numLinks)])

In [22]:
numNodes

416

In [23]:
from collections import defaultdict

node_neighbors_dict = defaultdict(list)

for node in range(numNodes):
    for link in Anaheim_links:
        if node == int(link[0]):
            node_neighbors_dict[str(node)].append(int(link[1]))

In [27]:
node_neighbors_dict['10']

[338, 362]

In [30]:
with open('./benchmark_data/Anaheim_trips.txt') as Anaheim_trips:
    Anaheim_trips_lines = Anaheim_trips.readlines()

In [39]:
numZone = int(Anaheim_trips_lines[0].split(' ')[3])

In [40]:
od_pairs = []
for i in range(numZone+1)[1:]:
    for j in range(numZone+1)[1:]:
        if i != j:
            od_pairs.append([i, j])

In [41]:
od_pairs

[[1, 2],
 [1, 3],
 [1, 4],
 [1, 5],
 [1, 6],
 [1, 7],
 [1, 8],
 [1, 9],
 [1, 10],
 [1, 11],
 [1, 12],
 [1, 13],
 [1, 14],
 [1, 15],
 [1, 16],
 [1, 17],
 [1, 18],
 [1, 19],
 [1, 20],
 [1, 21],
 [1, 22],
 [1, 23],
 [1, 24],
 [1, 25],
 [1, 26],
 [1, 27],
 [1, 28],
 [1, 29],
 [1, 30],
 [1, 31],
 [1, 32],
 [1, 33],
 [1, 34],
 [1, 35],
 [1, 36],
 [1, 37],
 [1, 38],
 [2, 1],
 [2, 3],
 [2, 4],
 [2, 5],
 [2, 6],
 [2, 7],
 [2, 8],
 [2, 9],
 [2, 10],
 [2, 11],
 [2, 12],
 [2, 13],
 [2, 14],
 [2, 15],
 [2, 16],
 [2, 17],
 [2, 18],
 [2, 19],
 [2, 20],
 [2, 21],
 [2, 22],
 [2, 23],
 [2, 24],
 [2, 25],
 [2, 26],
 [2, 27],
 [2, 28],
 [2, 29],
 [2, 30],
 [2, 31],
 [2, 32],
 [2, 33],
 [2, 34],
 [2, 35],
 [2, 36],
 [2, 37],
 [2, 38],
 [3, 1],
 [3, 2],
 [3, 4],
 [3, 5],
 [3, 6],
 [3, 7],
 [3, 8],
 [3, 9],
 [3, 10],
 [3, 11],
 [3, 12],
 [3, 13],
 [3, 14],
 [3, 15],
 [3, 16],
 [3, 17],
 [3, 18],
 [3, 19],
 [3, 20],
 [3, 21],
 [3, 22],
 [3, 23],
 [3, 24],
 [3, 25],
 [3, 26],
 [3, 27],
 [3, 28],
 [3, 29],
 [3,

In [6]:
with open('../temp_files/link_length_dict_Anaheim.json', 'r') as json_file:
    link_length_dict = json.load(json_file)

In [7]:
link_label_dict = zload('../temp_files/link_label_dict_Anaheim_network.pkz')
link_label_dict_ = zload('../temp_files/link_label_dict_Anaheim_network_.pkz')

In [8]:
keyLink = str(link_label_dict_['1->2'])
link_length_dict[keyLink]

6.0

In [9]:
keyLink = str(link_label_dict_['2->6'])
link_length_dict[keyLink]

5.0

In [10]:
keyLink = str(link_label_dict_['6->2'])
link_length_dict[keyLink]

5.0

In [11]:
# compute length of a route
def routeLength(route):
    link_list = []
    node_list = []
    for i in route.split('->'):
        node_list.append(int(i))
    for i in range(len(node_list))[:-1]:
        link_list.append('%d->%d' %(node_list[i], node_list[i+1]))
    length_of_route = sum([link_length_dict[str(link_label_dict_[link])] for link in link_list])
    return length_of_route

In [12]:
routeLength('1->2->6->2')

16.0

In [13]:
with open('../temp_files/path-link_incidence_Anaheim.txt', 'w') as the_file:
    for od in od_pairs:
        origi = od[0]
        desti = od[1]
        the_file.write('O-D pair (%s, %s):\n'%(origi, desti))
        route_list = []
        if desti in node_neighbors_dict[str(origi)]:
            route_list.append('%s->%s\n'%(origi,desti))
        for i in node_neighbors_dict[str(origi)]:
            if i in node_neighbors_dict[str(desti)]:
                flag = [origi, i, desti]
                if len(set(flag)) == len(flag):
                    route_list.append('%s->%s->%s\n'%(origi,i,desti))
            for j in node_neighbors_dict[str(i)]:
                if j in node_neighbors_dict[str(desti)]:
                    flag = [origi, i, j, desti]
                    if len(set(flag)) == len(flag):
                        route_list.append('%s->%s->%s->%s\n'%(origi,i,j,desti))
                for k in node_neighbors_dict[str(j)]:
                    if k in node_neighbors_dict[str(desti)]:
                        flag = [origi, i, j, k, desti]
                        if len(set(flag)) == len(flag):
                            route_list.append('%s->%s->%s->%s->%s\n'%(origi,i,j,k,desti))
                    for l in node_neighbors_dict[str(k)]:
                        if l in node_neighbors_dict[str(desti)]:
                            flag = [origi, i, j, k, l, desti]
                            if len(set(flag)) == len(flag):
                                route_list.append('%s->%s->%s->%s->%s->%s\n'%(origi,i,j,k,l,desti))    
                        for m in node_neighbors_dict[str(l)]:
                            if m in node_neighbors_dict[str(desti)]:
                                flag = [origi, i, j, k, l, m, desti]
                                if len(set(flag)) == len(flag):
                                    route_list.append('%s->%s->%s->%s->%s->%s->%s\n'%(origi,i,j,k,l,m,desti))  
                            for n in node_neighbors_dict[str(m)]:
                                if n in node_neighbors_dict[str(desti)]:
                                    flag = [origi, i, j, k, l, m, n, desti]
                                    if len(set(flag)) == len(flag):
                                        route_list.append('%s->%s->%s->%s->%s->%s->%s->%s\n'%(origi,i,j,k,l,m,n,desti))
                                for o in node_neighbors_dict[str(n)]:
                                    if o in node_neighbors_dict[str(desti)]:
                                        flag = [origi, i, j, k, l, m, n, o, desti]
                                        if len(set(flag)) == len(flag):
                                            route_list.append('%s->%s->%s->%s->%s->%s->%s->%s->%s\n'%(origi,i,j,k,l,m,n,o,desti))
                                    for p in node_neighbors_dict[str(o)]:
                                        if p in node_neighbors_dict[str(desti)]:
                                            flag = [origi, i, j, k, l, m, n, o, p, desti]
                                            if len(set(flag)) == len(flag):
                                                route_list.append('%s->%s->%s->%s->%s->%s->%s->%s->%s->%s\n'%(origi,i,j,k,l,m,n,o,p,desti))
                                                for q in node_neighbors_dict[str(p)]:
                                                    if q in node_neighbors_dict[str(desti)]:
                                                        flag = [origi, i, j, k, l, m, n, o, p, q, desti]
                                                        if len(set(flag)) == len(flag):
                                                            route_list.append('%s->%s->%s->%s->%s->%s->%s->%s->%s->%s->%s\n'%(origi,i,j,k,l,m,n,o,p,q,desti))

        route_length_list = np.array([routeLength(route) for route in route_list])
    
        refined_route_list = []
        
        if len(route_list) <= 20:
            refined_route_list = route_list
        else:
            refined_idx_list = [np.argsort(route_length_list)[i] for i in range(20)]
            refined_route_list = [route_list[idx] for idx in refined_idx_list]
                
        for route in refined_route_list:
            the_file.write(route)
        the_file.write('\n')